In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [44]:
df = pd.read_json("/content/train.json")
df.head(5)

# df["description"]

df["description"][4]

'Spacious 1 Bedroom 1 Bathroom in Williamsburg!Apartment Features:- Renovated Eat in Kitchen With Dishwasher- Renovated Bathroom- Beautiful Hardwood Floors- Lots of Sunlight- Great Closet Space- Freshly Painted- Heat and Hot Water Included- Live in Super Nearby L, J, M & G Trains !<br /><br />Contact Information:Kenneth BeakExclusive AgentC: 064-692-8838Email: kagglemanager@renthop.com, Text or Email to schedule a private viewing!<br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><p><a  website_redacted '

We can see here that there are html tags in the text which is not required

In [45]:
#Removal of outlier
upper_bound = np.percentile(df["price"].values, 99.9)
df_filtered = df[df["price"] <= upper_bound]

In [46]:
df_filtered["date"] = pd.to_datetime(df_filtered["created"]).dt.date
df_filtered["year"] = pd.to_datetime(df_filtered["date"]).dt.year
df_filtered["month"] = pd.to_datetime(df_filtered["date"]).dt.month
df_filtered["day"] = pd.to_datetime(df_filtered["date"]).dt.day
df_filtered["hour"] = pd.to_datetime(df_filtered["date"]).dt.hour
df_filtered["weekday"] = pd.to_datetime(df_filtered["date"]).dt.weekday
df_filtered["quarter"] = pd.to_datetime(df_filtered["date"]).dt.quarter
df_filtered.head(5)

<ipython-input-46-200f75a3fb14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["date"] = pd.to_datetime(df_filtered["created"]).dt.date
<ipython-input-46-200f75a3fb14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["year"] = pd.to_datetime(df_filtered["date"]).dt.year
<ipython-input-46-200f75a3fb14>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,price,street_address,interest_level,date,year,month,day,hour,weekday,quarter
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,...,2400,145 Borinquen Place,medium,2016-06-16,2016,6,16,0,3,2
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,...,3800,230 East 44th,low,2016-06-01,2016,6,1,0,2,2
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,...,3495,405 East 56th Street,medium,2016-06-14,2016,6,14,0,1,2
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,...,3000,792 Metropolitan Avenue,medium,2016-06-24,2016,6,24,0,4,2
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,...,2795,340 East 34th Street,low,2016-06-28,2016,6,28,0,1,2


PREPROCESSING
- Removal of html tags
- Convert text to lowercase


In [47]:
def remove_html_tags(description):
    description = re.sub(r'<[^>]+>', '', description)
    description = re.sub(r'\s+', ' ', description)
    return description.strip()


df_filtered['processed_description'] = df_filtered['description'].apply(remove_html_tags)

df_filtered["processed_description"] = df_filtered["description"].apply(lambda desc: desc.lower())

<ipython-input-47-9a9272e20c40>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['description'] = df_filtered['description'].apply(remove_html_tags)
<ipython-input-47-9a9272e20c40>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["description"] = df_filtered["description"].apply(lambda desc: desc.lower())


'brand new gut renovated true 2 bedroomfind yourself and your home in the center of it all. steps from grand central station, at the epicenter of manhattan, the centra combines convenience and luxury to create a perfectly balanced living experience. offering newly renovated over sized apartment layouts.full time doormanelevatornewly renovated hallwayslaundry in buildingon-site parking garagei operate with the utmost care and integrity. the client is my #1 priority. contact me for a viewing of the great apartment, i\'m more than confident we\'ll find a place for you to call home.call/text keon: email: if you require a move within 30 days write "urgent" in the subject email or text message to be taken with high priority.one month free - net effective rent listed<a website_redacted'

Tokenization : Split the text into words (tokens) for easier processing.

In [49]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# Load spaCy's pre-trained model (small model for efficiency)
nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def spaCy_tokenize(description):
    doc = nlp(description)
    return doc # Return the spaCy Doc object instead of just token texts

def remove_stopwords_and_lemmatize(doc): # Accept spaCy Doc object
    processed_words = []
    for token in doc:
        if token.text not in stop_words and not token.is_punct: # Access token.text for stop word check
            processed_words.append(token.lemma_)
    return " ".join(processed_words)
# To be removed after testing.
df_filtered = df_filtered.iloc[:100]
df_filtered['processed_description'] = df_filtered['processed_description'].apply(spaCy_tokenize) # Store Doc objects
df_filtered['processed_description'] = df_filtered['processed_description'].apply(remove_stopwords_and_lemmatize)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                          description  \
4   (spacious, 1, bedroom, 1, bathroom, in, willia...   
6   (brand, new, gut, renovated, true, 2, bedroomf...   
9   (*, *, flex, 2, bedroom, with, full, pressuriz...   
10  (a, brand, new, 3, bedroom, 1.5, bath, apartme...   
15  (over, -, sized, studio, w, abundant, closets,...   

                                processed_description  
4   spacious 1 bedroom 1 bathroom williamsburg!apa...  
6   brand new gut renovate true 2 bedroomfind home...  
9   flex 2 bedroom full pressurized wall**looke pe...  
10  brand new 3 bedroom 1.5 bath apartmentenjoy fo...  
15  sized studio w abundant closet available immed...  


In [54]:
print(df_filtered['processed_description'].head())

4     spacious 1 bedroom 1 bathroom williamsburg!apa...
6     brand new gut renovate true 2 bedroomfind home...
9     flex 2 bedroom full pressurized wall**looke pe...
10    brand new 3 bedroom 1.5 bath apartmentenjoy fo...
15    sized studio w abundant closet available immed...
Name: processed_description, dtype: object
